In [ ]:
from google.colab import drive
drive.mount('/content/drive')
     
Mounted at /content/drive

import sqlite3
     

db_path = '/content/drive/MyDrive/eng_subtitles_database.db'
     

conn = sqlite3.connect(db_path)
cursor = conn.cursor()
     

import sqlite3
import pandas as pd
     
Exploring the Database Structure Before proceeding with data cleaning, it is essential to understand the structure of the subtitle database. This step helps us identify the available tables and their respective columns, ensuring that we extract relevant data for processing.

First, we retrieve the names of all tables within the SQLite database. Then, we inspect the schema of the zipfiles table to examine the column names and data types. This allows us to understand the structure and content of the stored subtitle data.

Next, we load the zipfiles table into a Pandas DataFrame to get an initial overview of the data. We use .head() to display the first few records and .info() to check for data types and missing values. This preliminary analysis will guide us in designing appropriate data cleaning steps.


cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)
     
[('zipfiles',)]

cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])
     
num
name
content

df1 = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df1.head()
     
num	name	content
0	9180533	the.message.(1976).eng.1cd	b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1	9180583	here.comes.the.grump.s01.e09.joltin.jack.in.bo...	b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2	9180592	yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd	b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3	9180594	yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd	b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4	9180600	broker.(2022).eng.1cd	b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...

df1.info()
     
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB
Sampling and Optimizing Data for Efficient Processing

Given the potentially large size of the subtitle dataset, working with the entire dataset may be computationally expensive. To improve efficiency while maintaining representativeness, we take a random sample of 30% of the data using sample(frac=0.30, random_state=42). This ensures consistency across multiple runs while reducing memory usage.

To further optimize memory consumption, we check the data type of the num column. If it is of type int64 and its maximum value is within the range of a 32-bit integer (2^31 - 1), we convert it to int32. This helps in reducing memory usage without compromising data integrity.

Finally, we verify the memory footprint before and after type conversion using .info(memory_usage="deep") and inspect the first few rows with .head(). These steps ensure that our dataset is both efficient and suitable for further cleaning and analysis.


selected_df = df1.sample(frac=0.30, random_state=42)
selected_df.info(memory_usage="deep")
     
<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      24749 non-null  int64 
 1   name     24749 non-null  object
 2   content  24749 non-null  object
dtypes: int64(1), object(2)
memory usage: 551.0 MB

if selected_df['num'].dtype == 'int64' and selected_df['num'].max() < 2**31:
    # Reduce num to int32 if safe (avoid overflow)
    selected_df['num'] = selected_df['num'].astype('int32')
     

selected_df.info(memory_usage="deep")
     
<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      24749 non-null  int32 
 1   name     24749 non-null  object
 2   content  24749 non-null  object
dtypes: int32(1), object(2)
memory usage: 550.9 MB

selected_df.head()
     
num	name	content
17262	9251120	maybe.this.time.(2014).eng.1cd	b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x89\x9a\x...
7294	9211589	down.the.shore.s01.e10.and.justice.for.all.(19...	b'PK\x03\x04\x14\x00\x00\x00\x08\x007\x8f\x99V...
47707	9380845	uncontrollably.fond.s01.e07.heartache.(2016).e...	b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x8f\x19\x...
29914	9301436	screen.two.s13.e04.the.precious.blood.(1996).e...	b'PK\x03\x04\x14\x00\x00\x00\x08\x00[\xaa\x99V...
54266	9408707	battlebots.(2015).eng.1cd	b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xf4<\x9aV...
Extracting and Decoding Subtitle Content

Subtitle data is often stored in compressed binary formats, making it necessary to extract and decode the content for further processing. In this step, we define a function decode_method() to handle the extraction of subtitle text from ZIP-compressed binary data.

The function works as follows:

It takes binary data as input, decompresses it using the zipfile module, and reads the extracted subtitle file.
Since ZIP files can contain multiple files, we assume that each archive contains only one subtitle file and read its contents.
The extracted content is then decoded using the latin-1 encoding to preserve special characters.
We apply this function to the content column of our dataset, creating a new column extracted_content that holds the decoded subtitle text. Once extraction is complete, we remove the original content column to free up memory.

Finally, we check memory usage before and after dropping the column to ensure efficient data handling. This step prepares our dataset for further cleaning and analysis.


import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text
     

selected_df['extracted_content'] = selected_df['content'].apply(decode_method)
     

selected_df.info(memory_usage="deep")
     
<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   num                24749 non-null  int32 
 1   name               24749 non-null  object
 2   content            24749 non-null  object
 3   extracted_content  24749 non-null  object
dtypes: int32(1), object(3)
memory usage: 1.9 GB

selected_df = selected_df.drop('content', axis=1)
selected_df.info(memory_usage="deep")
     
<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   num                24749 non-null  int32 
 1   name               24749 non-null  object
 2   extracted_content  24749 non-null  object
dtypes: int32(1), object(2)
memory usage: 1.4 GB

selected_df.head(3)
     
num	name	extracted_content
17262	9251120	maybe.this.time.(2014).eng.1cd	ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...
7294	9211589	down.the.shore.s01.e10.and.justice.for.all.(19...	1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ...
47707	9380845	uncontrollably.fond.s01.e07.heartache.(2016).e...	1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin...

# extract text

import re

def clean_text(text):
    text = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\r\n', '', text)
    text = re.sub(r'\r\n', ' ', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text
     

selected_df['extracted_content'] = selected_df['extracted_content'].apply(clean_text)
     

# selected_df.head()
selected_df.info(memory_usage='deep')
     
<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   num                24749 non-null  int32 
 1   name               24749 non-null  object
 2   extracted_content  24749 non-null  object
dtypes: int32(1), object(2)
memory usage: 612.4 MB

# Assuming your DataFrame is named 'selected_df'
new_df = selected_df.copy()  # Create a copy to avoid modifying the original

# Rename columns
new_df = new_df.rename(columns={'num': 'num', 'name': 'name', 'extracted_content': 'content'})
     

new_df.info(memory_usage='deep')
     
<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      24749 non-null  int32 
 1   name     24749 non-null  object
 2   content  24749 non-null  object
dtypes: int32(1), object(2)
memory usage: 612.4 MB

new_df = new_df.reset_index(drop=True)
new_df.head()
     
num	name	content
0	9251120	maybe.this.time.(2014).eng.1cd	Watch any video online with OpenSUBTITLES Free...
1	9211589	down.the.shore.s01.e10.and.justice.for.all.(19...	Oh I know that its getting late but I dont wan...
2	9380845	uncontrollably.fond.s01.e07.heartache.(2016).e...	Timing and Subtitles by The Uncontrollable Lov...
3	9301436	screen.two.s13.e04.the.precious.blood.(1996).e...	ethereal music apiOpenSubtitlesorg is deprecat...
4	9408707	battlebots.(2015).eng.1cd	Chris Oh no not the Minibots yelling Oh You le...

new_df['content'][0]
     
'Watch any video online with OpenSUBTITLES Free Browser extension osdblinkext It couldve been just another summer But as I set foot on the sand that summer suddenly felt different Like it was going to be the summer that would change my life The summer of freedom The summer of endless possibilities The summer of Ooh aah Ooh oh Oh oh ooh That was the summer of you and me Youre quite the dancer Why did you stop Come on Keep dancing Whatever Im kidding Dont get mad Huh What Hey Im just going to get my towel What Stop that You thought I was gonna kiss you No Excuse me I wanna kiss you but not just yet What do you mean not yet Only when youre my girl What do you mean your girl My girlfriend Miss As if You wish And dont call me miss Dont pretend to be a gentleman when youre clearly not So what should I call you Rude Snob Bitch And you Douche Handsome Conceited Just like you Huh Jerk Exactly your type Leave me alone Steph Aha Steph Ill just call you Tep Remove the S and the F By the way Im Tonio Will you still be here tomorrow Dont leave yet Im going to court you I chose to walk away from you But fate had a different plan Councilor Were teaching basic English literacy Well be teaching the children how to read and write in English Is that so Yes How long will this program run If its okay the entire summer I thought I could escape you But you somehow found me again Tep What are you doing here You couldnt resist me huh So youre courting me instead Excuse me Im not here to court you Oh So youre here to be courted Yeah No Uncle Tonio shes the one who will teach us this summer Dont believe that story Thats just her excuse to get me to date her Gramps Uncle Erning Aunt Elma this is Tep My suitor Approved Approved You passed Mmm I dont approve Excuse us Well be going now All right my dear Thank you sir Well go ahead Excuse me please Oh no Tep Tep are you okay I said Excuse me Were not yet a couple and were already fighting He has a point Oh Tonio Seventy Snacks Seven Seven Silvery Silvery Sheep Sheep Do you know what that is in Filipino No Filipino Seventyseven silvery sheep Why say it in English if you can say it in Filipino Because they need to learn how to read write and speak properly in order to make a living Why are you here anyway Tep they dont need to learn that to make a living Instead of seventyseven slippery What What did you say Instead of teaching them that you should just teach them how to swim In a sinking boat English proficiency wont save you Youll be dead As youre well aware thats not part of the lesson So why are you disturbing us So irritable Why are you so hotheaded Maybe we should just go swimming Sounds fun What Hey kids Do you wanna go swimming Yes Swimming Swimming Swimming Okay You really want to swim Do you wanna swim I do Lets go You first So thats how you want it Swimming Swimming Hey Tonio Swimming swimming Put me down Swimming swimming So this is what you want huh Wait a minute What now Wait why are you hugging me You want me badly huh Of course not What now huh Hey stop it What now Come on Take this Tep What happened Oh no Uncle Tonio stepped on a sea urchin You saved me you cared for me and you made me feel alive Tep have you tried this Ive never seen caramel flan used as a bread filling Once you taste this youll surely fall in love with me Really Yeah Who would have thought You may be unlucky at finding a job but youre very lucky at finding a girlfriend You look good together You heard that We look good together That means were a couple Mmm That quick Why not But you havent officially courted me yet So can I court you Well you did save me a while ago Of course I would save you Id rather get hurt than see you get hurt So Can I court you now Mmm Fine sure Yes Ouch Whyd you do that Im okay Are you sure Here Okay Thank you Kiss Huh What kiss Im talking about the bread Oh Whatever Kiss You surprised me there Mmm Good Mmm Its good Told you so Slowly Mmhm Were almost there Okay Are we there yet Yeah Keep your eyes closed Mmhm One Two Three Wow Its beautiful Tonio You like it Yes You know this is my favorite spot in this town In fact this is the most beautiful thing Ive ever seen Mmm Until I saw you Nice Wow Youre just teasing me Its true Ive got one more surprise for you Theres more Yes Just look at me Okay Whats that Blow Blow Yeah Oh Its so cute Give me your hand I dont have money to buy you flowers so I got this instead Tep starting now I am courting you Okay Where did you buy this Its so pretty I made that You made this Yes Really You made this Yeah You want to see how I made it Sure Wow These are beautiful You made all of these Wow Youve been doing this for a long time My dad started this but I also developed a knack for it You know Tep once I have enough money from working as a seaman I want to set up a furniture shop That was my fathers dream Really Wow Ill be your first customer Wait question Why do you use old wood I dont like seeing these pieces of driftwood go to waste Most people see these as rotten old and useless But when you really stop to look at them thats when you will see their true beauty Unfortunately not many people do that A lot of people think these are junk They just throw them away like trash You captivated me through the beauty of your dreams and the kindness of your heart And slowly but surely I was falling for you Ah Sorry Sorry Does it hurt Sorry Every moment we spent together felt like happily ever after This wont take long Tep I will cook my special chicken skewers for you Tonio huh Hey why are you erasing it This is how you should do it T heart T Tonio loves Tep And vice versa Teptep loves Tonio I know you love me too right But I know you wont admit it Tep do you see it Mmm You like it I like it Lets go Lets go I didnt know where we were going but I was just happy to be wherever you were Hold on tight Oops Start paddling Are you serious Im just kidding Come on Okay Are you tired A little bit You Hey Steph how are you Steph Brother You okay Its her family Daddy what are you doing here I was suddenly given You surprised me a vacation leave Porky Hi Maam Hi Sir So hows your vacation This isnt a vacation This is an outreach program Wheres Abby Sleeping Asleep Steph whos that hunk with you The one you were holding hands with Is he your boyfriend No Stephanie introduce your friend to us will you Moms right Okay Tonio come here Ill introduce you Do I need to watch over you Really brother Tonio my family Good evening Good evening You know Tonio you should put a bench here so we have somewhere to sit Or maybe a swing Thatll be cool Hey Are you okay Why are you so quiet Was my family too much for you Oh no Your family is fun Im actually jealous I just wish I had a family like yours Happy and complete Its a shame Tep I cant introduce you to my mom She left us when I was just a kid I dont even think she knows I exist She wasnt happy with a simple life Thats why she left me and my dad Come here Tep Hmm What if this is it for me What if this is all I can be Can you live with that I knew at that moment I wanted to be part of your world and I wanted you to be in mine Hey you You said you would only kiss me once Im your girl right Youre something else Here Okay Tomorrow well meet up here again So I can give you my answer Why not give me an answer now Because once I give you my answer theres no turning back Its forever So you need to seriously think about it You need to be sure Can you love me forever Of course If you want I can even love you forever and forever and forever Mmm What about you Can you love me forever Mmm And slowly our worlds were becoming one Ouch Wait Bad ant He got to kiss you first Mommy Steph Its so late Weve been waiting for you Come on lets eat Where were you Up on the hill Oh its beautiful there Mommy You should see it Okay Oh Mommy Mmm Mommy Yes How did you know that Daddy was the man youre going to love forever I tried to make our summer last forever But all seasons come to an end Thats the story that inspired my presentation In fairness thats so good Your pitch is fabulous Its beautiful I can feel it Your delivery is topnotch sis You think so Of course I feel it too No matter how many times Ive heard it Im still affected by Teptep and Tonios summer to remember Also known as the summer to forget Forget To forget Hello guys That was seven years ago Lets move on If I were in your shoes I wouldnt be able to move on until I found out what happened He didnt show up because he got on a ship Thats what his family told me There you have it You mean to tell me you never saw each other You never talked again No Not yet Actually theres no need for it You do need it For closure Sometimes not having closure is the closure Wow Who I am and what I have right now thats the reason why the things that didnt happen never took place We will let the emoticons tell the story How embarrassingly corny and clich What do you think It might get rejected Right down to the stupid names of Tonio and Teptep Really now Here she goes again Is there anything else Um Miss Monica thats all we Steph I gave you one chance to come up with a brilliant marketing campaign Are you pitching me a soap opera No maam Its for a web series Three minutes each With a cliffhanger at the end of each webisode And to determine how the next webisode will begin well have a text poll using Kakao emoticons Uh for example if boy and girl will meet again how will the girl feel A angry B surprised Or C very angry Text the emoticon of the reaction you want to see in our next webisode How is product usage depicted in your series Your story was set seven years ago During that time KakaoTalk was not even in existence Miss Monica this is just the back story Do you really think that this is what the target market would like Dump the entire thing Give me something better Dont waste my time or the clients time Catch my drift Mmhm Here you go a gift from Aunt Steph Wow Thank you Youre welcome Share that okay That one Put that here Wheres the sauce Its here No its for the chicken Mommy heres your pork buns Ill just eat that tomorrow You know all of you need to learn how to save Your fathers been out of work for many years We have so many payables Bills rent your tuition The tuition of the kids Mommy relax It would be good if the businesses you got into were actually profitable And do you think youll get rich by selling perfumes and makeup Thats not even enough for my grandkids And you Get your dad Its time to eat Come on lets get this ready Okay Dont mind your mom Its enough that you bring food for us In fact she really likes the steamed buns that you bring home Did you leave some for me Of course Daddy Ill forget everything else before I forget your steamed buns Ouch ouch Oh Daddy Are you okay Where does it hurt Its fine Im okay Im okay I promise you Daddy once I get promoted and thats very soon Ill take you to the US for medical care Im fine I dont need all of that Daddy I want you to be able to walk again Steph Ive been in and out of therapy for seven years Weve even prayed in Manaoag so many times I think its impossible No Nothing is impossible You will be able to walk again Okay I will try to walk again as long as you try to fall in love again Well maybe Im already in love With whom Secret Howard our Skype date with the band is next week Uhhuh Yes actually theres to members of the entertainment press that are gonna be there From Taiwan Malaysia Singapore Vietnam and Hong Kong Sit properly and use your chopsticks Uhhuh Babes hang up the phone Eat sushi with me I cant I need to finish this now I wasnt able to work on this because you pissed me off earlier Yes Howard Of course I can take them to Boracay and Palawan for some rest and relaxation Mmhm Bye Howard Babe youre so naughty Youre lucky youre cute What do you expect You force me to learn to use chopsticks You used to find it sweet when I use my hands to feed you I want you to learn how to use chopsticks because I love Japanese food What will happen when we go on a date Youre gonna use your bare hands to hold the sushi Sorry Okay Have you thought of what kind of business you would like to invest in Yes babes I talked to Uncle Erning Hes going to request a business permit for the furniture shop Furniture shop What kind of furniture shop Driftwood Thats always been my dream Driftwood Yeah We have a lot of those back home That is not going to fly So babes what business will I set up Babe first of all its Babe Singular Theres only one me And also to answer your question you have to invest in something both trendy and relevant Whats that A bar A highend bar You know the Manila party scene is booming right now Ants the Italian you took care of left you million in his will But that million will easily vanish if you dont take care of it properly Actually its not even million anymore Because you bought this house Then you also bought one for your family Then you bought a car I just dont want you to invest in something that will surely fail Babe Ive been with you six months now When have I ever been wrong Hmm So much drama Okay fine Mmm Youre so dramatic Ah Miss Monica Um you asked me how we can make the national campaign a success What if we do a Metro Manila campus tour to see if our target market will like the web series We will set up booths to provide WiFi to those who have downloaded the app Youve got to be kidding me Steph The client has done that before I know maam but Do not give me rehash crap and say that its improvement Youve been asking me about a promotion With these ideas how can I promote you Anyway I have a more urgent project for you to work on I have a new client who wants to open a highend bar and I want you to personally take care of his account I want you to do the PR of the bar and its owner Okay And if you do well maybe Ill give you the vacant Account Executive position Really I said only if you do well then maybe I might promote you But this is confidential Yes maam The bar owner is my boyfriend Ah Mmhmm You see I met Anton on a cruise ship a few months ago Hes not used to how it is to live in my world Now your job is to transform him so that he can mingle with a highend crowd which is the target market of the bar hes opening He speaks a little bit of Italian and English is not his first language Its your job to teach him English as well Yes maam Can I trust you with this Yes maam Dont mess this up or else Catch my drift Waiter Im on my way Waiter Hi sir Im sorry to keep you waiting Im Stephanie Asuncion Sir Anton right Anthony actually Pleasure to meet you May I take this seat Sure Thank you All right This initial meeting is basically for us to discuss your vision and goals for the business you hope to launch Miss Monica informed me that you wanted to open a bar Are we talking about a restobar or is this a club or lounge Oh yeah Im sorry Im sorry I was talking my head off I forgot that English is not your first language Uh did you understand any of that I understood actually And English isnt just my first language Its my only language Hi honey Uh this is my wife Gina and this is A very stupid person Youre not my boss boyfriend are you He better not be or Im gonna kill him Honey Im not cheating on you Im telling you the truth this time I promise This time So you admit that youve been lying Stupid Steph Wheres my cell phone Okay Ill follow up on table Ah Sir Im sorry Im sorry maam Sir wait a minute Hold on I have a lot to do maam Teptep Tep Teptep Uh Tep No no Im okay Thank you Just a thank you Uh Sorry Tep its Tonio Oh yeah Tonio Hello How are you Hello Im good Great even What are you doing here Well it was nice bumping into you but I have to go because I have a client whos waiting for me Bye Tep wait Yeah Lets talk for a while Im sorry I have to meet an important client I cant and I dont want to attend to you right now Bye Hello Hello Sir Anton Good afternoon Tep Is that you To Tonio Tep its been awhile Steph or Stephanie Stephanie Tep is fine Steph is too formal Well you are a client Sir Anton Sir Anton Tonio Its like we dont know each other Okay so Tonio How are you As I said earlier Im good Great Okay And you How have you been Im good What have you been up to since I last saw you I heard you became a seaman Yeah I worked on a cruise ship So thats where you met Miss Monica right Good for you Tep if youre uncomfortable just tell me okay Ill just ask Monica to find someone else who can help me Of course not Theres no reason for me to feel uncomfortable Right Dont worry Tonio Relax Mmm By the way I want to say sorry Sorry For what happened Why what happened What do you mean When When it was us Us Was there an us Back when we were together when we had an MU MU Hmm MU mutual understanding No no I think your MU stands for MisUnderstanding Gosh Sorry Sorry Tonio There was never an us Look if this makes you feel uncomfortable I can just talk to Miss Monica about this and tell her Oh no Im fine No problem MU your face MU my ass MU messed up You are one big mistake Tonio A huge mistake Huge mistake Wow Good day Uh is Tonio around Mmm Mmm I like you You got my vote Are you the new girlfriend And by the way Im Mama Mae Im Sir Tonios allaround helper In by day Out by night And you are Stephanie Asuncion Ah Stephanie Hey My sir is there Tonio you have a visitor Shes very beautiful we are so alike Mama Mae Tep Hello Have you been here awhile Sorry for making you wait No its okay Im used to waiting for you Im actually surprised you came Hi babes Its babe Ah Babe Is Steph there already Yeah shes here Im embarrassed cause I made her wait Are you good Dont worry I got this Okay bye I love you Oh Oh You should have told me Told you what I thought you were a couple Mama Mae There was never an us Right Tep Exactly Mmhmm Hmm My job is to present you to your public in the best light possible To communicate the right message about your business and about you Were doing this interview so I can get to know you and make everything clear To avoid confusion We dont need another MU MU Werent you Misunderstanding MisUnderstanding Oh Mmhmm So why do you want to put up a bar So that I can make Monica happy because You love her Yeah You love her Why What do you like about Miss Monica Yes I ask myself the same question Why do you love that woman I dont get it I actually get it Shes beautiful powerful and most of all shes rich What are you trying to say Exactly that And you What does she like about you Oh That That I get Its because my Sir Tonio Ah Maybe its your job Cause Miss Monica is known for dating accomplished men Where are you working Im currently jobless So youre not working now because youre with Miss Monica Thats why Im opening a business right Which you havent even thought about I know what I want Yes you do My rich boss What are you getting at I wasnt getting at anything Uh I just wanted to know If Im after Monicas money If Im just using your boss You said it not me I am not using Monica but thats what youre getting at The truth Because if I misunderstand you once more Ill look stupid again right You have that effect on me You make me look stupid Huh What I just want to do my job properly okay Job Is this your job To insult me If thats all youre gonna do lets just end this Okay Fine All right Thank you Youre welcome Wow I felt that She had a lot of resentment You know you two need to MU MU Make Up Mommy Hey sis Wheres Daddy What happened His sides started hurting all of a sudden then he turned pale So we rushed him here Mommy Mom are you okay Doc how is my husband Hes fine for now We are doing some tests and it seems he has stones in his gallbladder We need to operate on your husband What we have to do is schedule the procedure immediately Based on what Anton said you broke every single rule in public relations Im sorry Miss Monica Please dont fire me I ought to fire you but I wont Because Im giving this project to Liana Miss Monica I need this project It doesnt seem like you want it Miss Monica please dont take the project away from me Ill prove that I can do it that Im deserving of the promotion Please Miss Monica Please give me another chance Dont talk to me Talk to Anton Hes the client And as far as Im concerned he doesnt wanna see you He doesnt wanna have anything to do with you So good luck in trying to convince him to take you back What You got something to say Im too busy counting all of Monicas money right now Please forgive me Please give me another chance because I really need this job Here A peace offering And this too Mmm Thanks Tep Are you mad at me Cause your insults really hit home You know it was my decision to wait for you I made that choice Eventually I realized that I should be mad at myself Because I chose to be stupid Im sorry that I left you hanging You know I completely understand that you had to board a ship for work But to leave without saying a word You didnt talk to me or come and see me one last time I found that hard to accept And I waited for you I even told myself Youll explain everything to me because you love me But come to think of it I realized you never once told me you loved me So it was stupid of me to assume that you loved me Oh well I guess when someone courts you it doesnt mean he loves you I did love you Oh well the past is the past Lets leave it behind us Its been seven long years Anyway its better this way Because if we ended up together I might have been a hindrance to you Hmm You might not be as successful as you are now Tonio Im really sorry Im sorry for everything I said Its okay Were good Mmm High five Kiss Mmm Hi Im Stephanie Asuncion I will be your PR specialist And you are Tonio Anton Bugayong Anton It doesnt sound like me That name doesnt suit me Not yet but we will do something about that Just relax Thats the kind of bar you and Miss Monica want to set up And as the owner of the bar thats also the image youll need to project And the first step to achieving that is appearance management You are the brand ambassador of your business So just like your bar you have to be posh modern and cool Posh Modern Cool Indeed Thats the look were going for Ill take care of him Please sit Sir Okay Ill go ahead Mama Mae Tep What Where are you going Ive got work to do Ill be back later Dont worry I wont leave you high and dry unlike you Im leaving too And where are you going Im just going to buy some dried prunes These are killer outfits Why Mama Mae these are better No These will suit him better Look at them No way I wont wear any of those clothes I like these The action star look No no no no Youre not wearing that Youre wearing these These will look better on you No youre wearing this No no no Im wearing this Im wearing this No Dried prunes Just kidding Mmm So do I look good You need to improve your communication skills Being handsome isnt enough Oh Thats enough for me More than enough actually Its not enough for you So you think Im handsome Why Arent you Just making sure Hmm Hey Tep wait Tep Hold on hold on Am I so handsome that you cant help but kiss me Excuse me Im not gonna kiss you You need a facial Facial Ouch Thats gonna hurt If you can tell me in straight English why you dont want to wear these I wont force you to try them on Okay Go ahead I dont like because Because It doesnt my unlike You doesnt you like Unli unlike No Fine Give me that I like It doesnt It should be It dont right Stay still Sir Tep this really hurts Why am I doing this again Because you love Miss Monica remember You know as they say love hurts Wow Looking good Hmm So what do you think Do I look good Mmm Modern cool Whats the last one Posh Posh Posh it further Mama Mae Yeah Ill posh you up too Me too Mmhmm Lets posh you Mama Mae Ow Ow Im sorry It hurts We will be done in a bit Ow Ooh Ow My hand It hurts Sorry sorry Sir please hold still Look Yuck Dinner fork Good Salad fork Very good Mmm Okay Lets try it without the cheat sheet this time Go Whats this I thought the chopsticks were hard but this is harder There are so many of them Dont worry With practice youll be able to memorize all of these Tadaaa Yes Wow Whats that Mama Mae Flowers Im kidding Its food of course A dish made by Tonio Peppered pork Tonios specialty Wow Especially for you Wow really Tep what fork should we use to eat this For this Hmm Well use our hands Oh Mamae Lets use our hands I will use my feet Your feet Mmm Okay Im done Im so full Thank you You liked it Hmm delicious But I think we should take a walk to help digest the food Yeah walk around Take a walk around the whole house Go ahead I had no idea you knew how to cook back then When I worked on the cruise ship I was assigned to the kitchen Thats where I learned to cook Hmm So thats how Miss Monica fell for you by putting a spell on her food during her cruise Of course not Thats not even where we met We met in Orvieto Italy What In Italy Yeah I only spent one year on the ship then I hopped off I ended up working as a caregiver Then I met my employer He looks like your grandfather Thats why I felt at ease taking care of him Did you have a hard time taking care of him Not really but its the loneliness that will get you There arent many Filipinos there Plus Seor Luigis vineyard was in a provincial area Oh Last year his closest friends visited him Monica was with them But after a few days Seor Luigi passed away Monica was the one who helped me She really surprised me She didnt mind doing the work Washing the dishes Serving the funeral guests Really Miss Monica did Yeah Really She doesnt seem to be the type Hey shes not like that Thank you I think youre talking about a different girlfriend Tonio You just dont know Monica like I do So why did Miss Monica tell me that she met you on a cruise ship Maybe because thats where we bonded When Seor Luigi died he left me a lot of money And he left strict instructions for me to go on vacation So when Monica told me she was going on a cruise I went with her Thats where you got to know all of Monica Meaning thats where you meddled with Monicas mufflers Mama Mae so so what Im just worried about you I just dont want you getting hurt by Tonios answer because you know Mama Mae whatever Tonio says will have no effect on me anymore And obviously thats where they They first you know Right First what First time you meddled Meddled with The mufflers Whose Monicas My gosh What are you saying Ouch Disgusting Sorry My God Im sorry Ill just shut up and drink my coffee Hello Babes I mean Babe How are your mufflers I mean your Its Mama Maes fault You know you may not be feeling hurt but I am hurting for you Mmm Mmhm Its going well Im learning a lot Shes just a bit temperamental and strict Oh my God Anton is that you Yes Babes Thats me Ants Its me Not Thats me Sorry Babes uh Babe Do you like this I love it You look so hot Did Steph pick out those clothes for you Yes Steph is really good Im learning a lots from her Except for good grammar But you look so hot anyway Tep did you get the thing I sent you You sent me something Guys Theres a special delivery for Steph Hurry up I want to have some So sweet Caramel flan Sir Anton your card please Thank you so much The package just arrived This is too much Thank you Thank you so much Monica was so happy last night Sorry I wasnt able to drop by I had to wire money to Grandpa For his medication Its okay Thank you for the food you sent And that other thing I sent Has it arrived Other thing Theres more Flowers for Steph Is it there yet Excuse me I have flowers for Miss Monica T Valencia Tep are they pretty Do you think Monica will like them What if we use stickers to tell the web series that I previously pitched Why do we have to burden the client with creating a new product Thats not part of their directives However I would like to congratulate you on the work youve done on Anton Major improvement Thank you But its not enough His basic English still needs work Mmm will he be ready by tomorrow Ill make sure of that Miss Monica It is our first time going public tomorrow and I dont want him to make a fool out of himself Mmhm Maam thank you for attending our event No problem How about you Steph Will you donate your hair Of course maam Later Have a seat Thank you Thank you sir Wishing youre here Is my English correct Oh your kindaex misses you Im not the one he likes remember So you admit it Youre still hoping that he likes you I knew it No No I didnt mean it that way Sorry Im at an event Wait Ill just text back But well see each other later right Go girl Meet with him You know you want to Excuse me No I dont want to Come on We wont judge you Youre just friends arent you Friends Friends Hi Sorry Im late I couldnt use my car today so I took a taxi Then he dropped me off at the wrong entrance So Lets review before Miss Monica gets here Lets go When did you get a haircut Earlier at the event Come on Lets get a table Wow Hmm Tep Wow Steph Oh Sir Andrew Hello Tep whos that Thats Sir Andrew One of my bosses Hes the ex of Wow In English please The guy who broke Monicas heart Hey Sounds good In straight English I may add Of course Its Miss Monica Hello Miss Monica Im here with Sir Anton Okay Are there a lot of people Um no not really Miss Monica Sir Andrew is here Huh What is he doing there I think hes on a date Um okay Well Im still in a client meeting right now so I wont be able to make it to dinner What Can I just talk to Anton please Okay She says she cant make it Hello Babes Its Babe Singular remember You told Steph to set this up I prepared for our date and now you cant make it I know but I have an important client meeting More important than me Look I dont want to fight about this okay Lets just talk about it tomorrow Just try to understand Fine I love you Okay bye Are you okay Are you angry No Just hungry Lets eat Tep Lets order You know Tep I dont understand Monica She dislikes it when I call her Babes but she keeps calling me Ants Singular Theres only one me You full Super Tep thank you for all your help Its just too bad Monica didnt see what youve worked so hard on What you have worked hard on Thank you Oh you have a meeting tomorrow with MTVs friend who wants to invest in your bar Why not have your date with MTV after the meeting If you want I can make reservations for you Tep what if I set up a furniture shop instead Thats what I really want anyway Its just Monica who thinks its a waste of money Huh Its not a waste of money Youre very good at that I promise you once your shop opens up Ill be your first customer Really Sure Ill even do PR work for you for free Really Uhhuh You know I think you just need to show your work to Miss Monica Im sure shell be impressed Why not bring her to your province Oh Why are you still up Im just editing a video presentation for a client Mmm Uh by the way who drove you home My new client He looks like Like Tonio Mmmm That was Tonio Mommy Hes your client My God Steph Hes nothing but trouble You need to get away from him while you can Unbelievable Go to the province with you What Why Ill introduce you to my family Ill also show you my woodwork I want to prove to you that setting up a furniture shop wont be a waste of money I just cant see myself running a highend bar Is this because I missed dinner last night So youre backing out Is that it Monica Im not interested in setting up a bar Then study up on it Why do you keep insisting on this furniture thing Monica youre my girlfriend but you dont seem to have faith in me Unlike Steph who has complete faith in me Ants sorry I have faith in you Okay Im gonna reconsider that furniture thing But I cant go to the province with you Maybe next time okay Move my meeting with Mr Reyes to pm and tell him Im on my way okay Yes maam Miss Monica Yes Can I ask you something Okay Make it fast because Im busy What I saw your car outside the restaurant last night Why didnt you come in Follow me Actually I kind of feel guilty about it I just wish life was less complicated When I met him in Italy I was still heartbroken over Andrew So when Anton told me he wanted to come with me on the cruise I was okay with it Normally I wouldnt have allowed it because I barely knew him But you know he made me laugh He made me feel beautiful and special Have you ever felt that way Steph His rural attitude His corny humor Even his bad grammar All of that was so wonderful That was back then When we were still in paradise But not now when were back in the real world Its just better to introduce him to family and friends as a fellow passenger that I met on a cruise ship Theres no way that I would tell them Im in a relationship with a seaman Turned caregiver Its embarrassing He had an honorable job Steph dont be so naive After all were in the world of PR And thats not the point This is the first guy who is willing to do anything for me who is willing to change for me Thats why I really want this relationship to work But you confronting me like this and planting seeds of hope in his furniture shop Miss Monica if you could just give Anton a chance I did I told him I would reconsider But only after Ive been proven wrong That its not going to be a waste of money Catch my drift Steph I think this is going beyond what Miss Monica is asking of you This is girlfriendlevel effort for a guy you claim to be just a friend Steph Im just concerned for you This might be another MU for you A MisUnderstanding with Major Uncertainty Come on Lets grab some lunch Okay I might lose some weight Oh gosh Im super excited Im ready for the beach See Look at me I put on sunscreen Oh Show a little excitement I cant believe you Finally you get to introduce me to your family Mmmm Tep Youre in uniform What are you doing here Do we have a session No but I heard MTV cant go with you to the province to see your furniture Yeah Im kind of bummed out about it Mmm No no no Youre not allowed to mope Im here because Im going to make sure that MTV will still see your furniture So instead of her coming over to see your work your work will come to see her okay Ill do it Tep Yeah Mmm Huh Mmm Mmm Oh Ah Got it Yes Mmm Huh Theres Theres more No You still got food Give it here I want some more You finished it Mama Mae You might get fat Yes braised pork Tep Lets eat You can sleep in my room tonight Huh Oh My gosh So naughty What about you Ill sleep on the couch Ill be fine there Too bad You really look good together Hmm You should have heard him say There was never an us Theyre fooling each other Youre like a newlywed couple separated by sadness Thats not true No were not Oh yes you were You almost didnt board your ship remember Huh Uncle thats not true Tep might believe you Remember when you were in Manila You were drunk every night And every time you got drunk you sang Tep Tep Tep Tep Tep With matching tears Thats too much Tep might believe you Too much You were too drunk They even had to dunk your head in a drum of water just to get rid of your hangover Hey Steph got drunk too Hey You got drunk Tep Tonio you didnt know about that Because he didnt want to talk about Steph back then Okay Enough She reeked of booze Hyro Right Mom Wait So what if she got drunk I wasnt drunk That was when she came back to look for you The second time she came back Susan Three months later Tep did you really come back here Oh she came back Wait You tease them like Tonio doesnt have Monica She was even waiting for Hyro thats enough Dont you remember Mom You were the one Tep cried to Then her daddy came to pick her up Okay Stop now Stop now son Why Mom Can you just pass me the rice We have to eat everything okay Thats right It seems youre getting thinner Its very delicious This is what I missed from Tonios cooking This one is good Who cooked this Mama Mae why are you still up Im having a hard time sleeping because of this cold Here Drink this before someone else catches a cold Thank you Why are you still up My cold is curable Ill be able to sleep now But I dont think theres a cure for whatever youre feeling Kooks How are you Well Im okay Steph I would just like to remind you youre doing this for your future promotion Your past with Tonio it has nothing to do with it okay So please dont go back to being stupid okay I love you Two old friends meet again Wearin older faces Talk about the places theyve been Steph Two old sweethearts Get over it Who fell apart somewhere long ago Why do you have to find out when its already in the past How were they to know Someday theyd meet again And have a need for more than reminisin Maybe this time itll be lovin theyll find Maybe now they can be more than just friends Shes back in his life and it feels so right Maybe this time love wont end Yeah yeah yeah yeah Did you get drunk over me Did you come back for me Its the same old feeling Tonio Steph Lets go Back again Its the one they had way back when They were too young to know when love is real But somehow some things never change And even time hasnt cooled the flame Its burnin even brighter than it did before Its okay It got another chance and if they take it Ah Wow Just like newlyweds Maybe this time Itll be lovin theyll find Maybe now they can be more than just friends Shes back in his life and it feels so right Maybe this time love wont end Okay Ready One two three Go A piece of driftwood may look like nothing in the beginning But I always find beautiful in Oh Sorry Dads asking me to pick him up He says hes got our lunch I wont take long Ill just go get them Wait What about Hyro Ill be quick Um Lets continue Sure Sure Okay No you can stay there Yeah same spot Okay Game Okay One two three Go I dont just work with driftwood I also work with recycled metal Like this one And recycled wood Like this because Because Because Because there are things you cant just throw away Because you just cant forget about them Because you just cant let them go Tonio thats not whats written here It wasnt easy To walk away It was the most difficult decision you ever had to make But at that time It was the right thing to do You shouldnt be saying those things But what you thought youd lost Has come back And you take it back Because its precious to you Because you realize that you still love her You got hurt Tep I know There was never an us There will never be an us So please stop flirting with me Tep sorry Sorry if you think that I That Im flirting with you But Im not Its not like that I dont know what to do I dont know how I feel Its nothing Tonio You dont feel anything You shouldnt feel anything And I shouldnt feel anything Because there was never an us But we loved each other Tep I loved you Believe me I loved you Tonio You were my first love Everyone made fun of me Because we never became a couple but I went crazy over you Because every part of my body Was hurting sad Every part of my body was longing for you Thats why I came back here Because I thought you might be here Forgive me I got so drunk that night Thats why my dad had to pick me up He was tired and it was dark We got into an accident My dad was paralyzed because of my stupidity over you And to this day Im still paying the price for that stupidity But even after all the pain youve caused me I still love you Tonio I still love you You think youre the only one who suffered The night before we were supposed to meet your mother talked to me She told me I would only hinder you from reaching your dreams But if I really wanted you then I should take you but you could never go back to them It was a lot to take You were a coward I wasnt ready Tep You were a coward Admit it Tonio You were a coward Because I wasnt ready What life could I offer you How was I supposed to give you the life you were used to That was not the life I wanted A life with you was more than enough for me But not for me Because I love you Because I wanted to give you a good life I didnt want you to go through the same hardships I did You should have said that I should have heard that So that I could have told you that I can wait until you were sure until you were ready I would have waited no matter how long because I loved you Tonio You should have showed up I waited for you Tep please forgive me Because if I came to meet you theres no way I could have left you Tep Im sorry Coming back here was a mistake You already You already love someone else Steph Im so sorry All this time Ive done nothing but hurt you Please Tonio lets stop this Just tell me its over Steph Can you Can you please stop telling me That you love me Just tell me that its over Please Its over What You spent the weekend in the province with Steph Why didnt you tell me And whats all this talk about driftwood furniture Monica this is what I really want to do And you think youre gonna be successful with it Teptep thinks I can do it Even when we were younger she always believed in my dream Teptep Stephanie Yeah Steph You knew each other Even before That was a long time ago Its not important Why didnt you tell me There was nothing to tell Dont lie to me Monica thats in the past Its over We will be launching a web series with cliffhangers and have a text poll over Kakao to find out how the next webisode should begin Well be developing a new group of emoticons to tell the story Its going to be like an animated short I smell a promotion And to generate more buzz we are going to launch via a campus tour just like what you did before I like it Its fresh and innovative But what will the story of the short films be A love story Girl and boy meet one summer What do the letters A M stand for Anton and Monica But their love story was cut short that summer But a few years later they meet again So this time will Monica and Anton have their happy ending Okay I like it And its a go Congratulations Monica You came up with all that Yes I did And thats why shes the best Good job Monica Thank you Oh and by the way I would like to introduce to you our new project head Liana Mendoza our newest account executive Congrats Congrats Liana Thank you maam What I deserve it Deserve what The promotion you gave to Liana Excuse me Get out of my way Everything you presented were my ideas And you didnt even bother to tell them that they werent yours I was the one who put this together Not you I was the one who worked on all of them You worked on all of them including my boyfriend Dont you dare deny it I can see right through you I didnt do anything wrong But you wanted to You wanted to steal him from me I would never steal him from you I am not like you You wanted to go back to the province because you wanted to bring back the past Oh yes I know everything If thats the case you should know that its over Its all in the past It better be because hes my boyfriend now so stop being delusional You cant compete with me Dont even try You lying backstabbing boyfriendsnatching slut Really now This Coming from a shallow manipulative twofaced bitch I dont want to deal with this garbage Im not done yet Let go No you listen to me Before you call me a liar its time for you to come clean Do you really love Tonio Yes I love him No you dont Because if you truly love him you wouldnt force him to change just so he can be good enough for you Dont even try to And dont you dare call me a backstabber I never betrayed you I respected your relationship with Tonio So you have no right calling me a slut Youre fired No I quit Tep Babe This is not the right time for you to resign Stephanie Mommy I cant stand my boss anymore Shes so unfair Not only to me but also to Tonio Tonio What the hell Stephanie You almost quit school before Now youve quit your job Youre going to destroy yourself again over that man Youre letting him make a fool of you again Who else will suffer this time Havent you learned your lesson He already left you once He wouldnt have done so if you didnt talk to him If you didnt scare him into thinking that he would ruin my life But dont worry I wont let myself get fooled again I wont let myself get destroyed again I will not love Tonio again even if thats how I really feel I will do that Not because I think its wrong or because I might get hurt again I will do it Its because I dont want to give you more reason not to love me I dont want another seven years to go by That whenever you look at me all you see are my mistakes Here Tonio This will calm you Tonio Are you okay Are you feeling ill Maybe you shouldnt go to Monicas party Mama Mae I have to Monica is expecting me And Ive been waiting a long time for this So why dont you look too happy You look like youd rather go somewhere else Mama Mae Monica is my girlfriend I should be with her Its why I changed for her As the saying goes Change is good as long as you change for the better Thats in English This is whats right This is what should be You know I admire your determination to stay with Monica But if youre sticking to that You might be stopping yourself from experiencing the joy of true love Keep in mind Tonio determination is not the same as love Hi babe You look great Mmm Stand up straight How are you Um are you ready I want to introduce you to my friends Come on Anton Monica told me you plan to open a bar Have you picked out a location Were still looking for a place where we Actually Anton is still doing his research We already know that theres a lot of bars that are open right now So we wanna make sure that his stands out You know Anton he loves to take calculated risks Come on Anton I know you worked abroad So Im sure you know about taking risks Yeah In fact Ive been to many countries I started working as a Actually Anton has worked in so many places When I met him he was working for a big wine ranch in Italy And I thought he was done with it But you know once a businessman always a businessman Actually Im not sure if Im a good businessman at all But youre right But why Im used to taking risks Thats what I did when I jumped off a ship to become an illegal immigrant Oh he was a seaman before Thats interesting It was hard but a lot of people helped me Thats why I have so much respect for wealthy people like you So when I got rich I told myself that I should never feel ashamed about where my money came from Because that was a blessing to me a blessing given to me by a kind person who accepted me for who I was Youre really lucky then Thank you for the wonderful evening Please excuse me I have to go Uh excuse me Ill be back I think theyre having a fight Okay Anton Anton What was that all about You humiliated me in front of my guests I just defended myself Because I felt so small I didnt embarrass you Monica You embarrassed me I was just protecting you from them Protecting me Why Because theyre judgmental They didnt even mind my bad English and my being unsophisticated But you did Monica You think they werent judging you They were just being nice to you but they were already judging you in their minds But youre judging me to my face You couldnt even let me speak for myself Because youre afraid everything I say will embarrass you Oh my god Why are you being so juvenile about this Is this about Stephanie again Steph has nothing to do with this Well ever since you started working with her youve changed Isnt that what you want That I change for you You make it sound like change is such an awful thing Anton Tonio My name is Tonio This isnt easy for me Tonio This isnt easy for me either Monica To admit that Im not enough that Im so flawed in your eyes The pain I feel right now is the same as when my mother left me the same pain I felt when I had to leave Steph because I felt that I wasnt good enough for her Because I love you I let you change who I was thinking that once Im finally a good fit for you you might love me wholeheartedly But now I realize that will never happen We will never be a good fit for each other No matter how much I study English no matter how hard I try to fit in with your friends and no matter what clothes I wear Im still nothing but trash to you Im so tired Monica Are you saying that its over Im sorry Daddy Steph come here Steph its okay Its okay to get tired to get hurt Its even okay to make mistakes But you also have to forgive yourself Daddy Daddy Im so sorry Im so sorry for what happened to you Steph I never blamed you nor does your mom So stop thinking like that She just doesnt want you to get hurt again She knows that even though I got paralyzed youre the one with the deeper wound She just doesnt know how to help you But Daddy She still resents me for what happened Steph Steph Im sorry I didnt mean to I love you so much I love you too Mommy Stephanie Asuncion its time to spread our wings Lets just start our own firm Thats right You think we can do it Come on girl This will be fun Plus we already have our first client Really Hi Tep How have you been Oh yes maybe shes here already Guys Steph is here Hi Mr Stewart I want you to meet one of our partners in the PR company Its nice to finally meet you This is Steph Oh hello Its nice to meet you too Im glad you got to see the exhibit before it opens tomorrow Let me show you around Its right this way Sure Lets go All the pieces are locally made Mmhmm My business partner who also designs all the furniture he collects all the driftwood himself Lets go there Yeah theres another display over here Hey Teptep Pssst Teptep Why arent you answering Rude Snob Bitch Finally you smiled Hey shes looking for me Hmm Hey Your client is waiting for you Tep So this is yours Im impressed You finally fulfilled your dream Yes But its incomplete Without you What about Monica Were through Tep Id like to court you again I will make sure that you know That I love you so much Then now And forever and forever and forever In case its not clear all together now Tonio loves Teptep Or Teptep loves Tonio Tep Im ready I know I made you wait But Im here now Tep give me one more chance Please Of course Go Tonio Oh Oh Yes Oh Oops Youre heavy Sorry Two old friends meet again Hey Mmm Im still courting you and you already want to kiss me Oh sorry I thought were MU now MU Here we go again I hate MisUnderstandings MU Me and U Me and U Me and U Were a couple now Mmhmm So can I kiss you now Kiss me Mmhmm Fine Theres nothing I can do about it Its me and you now Okay Mmm What is it this time My parents are strict Your parents arent here And our friends are gone Shes smilin like she used to smile way back then Shes feelin like she used to feel way back when They tried but somethin kept them Waiting for this magic moment Oh Maybe this time itll be lovin theyll find Maybe now they can be more than just friends Oh Shes back in his life and it feels so right Maybe this time Maybe this time Maybe this time Love wont end Two old friends meet again Wearin older faces Talk about the places theyve been Two old sweethearts who fell apart somewhere long ago How were they to know someday theyd meet again And have a need for more than reminisin Maybe this time itll be lovin theyll find Maybe now they can be more than just friends Shes back in his life and it feels so right Maybe this time love wont end Yeah yeah yeah yeah Its the same old feeling back again Its the one they had way back when They were too young to know when love is real But somehow some things never change And even time hasnt cooled the flame Its burnin even brighter than it did before It got another chance and if they take it Maybe this time Itll be lovin theyll find Maybe now they can be more than just friends Shes back in his life and it feels so right Maybe this time love wont end Shes smilin like she used to smile way back then Shes feelin like she used to feel way back when They tried but somethin kept them Waiting for this magic moment Oh Maybe this time itll be lovin theyll find Maybe now they can be more than just friends Oh Please rate this subtitle at wwwosdblinkapcu Help other users to choose the best subtitles'

new_df.to_parquet("/content/drive/MyDrive/cleaned_subtitles.parquet", engine="pyarrow", index=False)
     
Storing Cleaned Subtitle Data

After extracting and processing the subtitle content, we perform a final verification by inspecting a sample of the cleaned data. We check the content column to ensure that the extraction and decoding steps were successful.

To enable efficient storage and retrieval, we save the cleaned subtitle dataset in Parquet format using the pyarrow engine. Parquet is a highly optimized columnar storage format that offers better compression and faster read/write performance compared to traditional formats like CSV.

By saving the cleaned data as cleaned_subtitles.parquet, we ensure that it is ready for further analysis, such as text preprocessing, keyword extraction, and search engine optimization. This marks the completion of the cleaning_subtitles.ipynb file, setting the foundation for enhancing search engine relevance using subtitles.



     


     